In [1]:
import torch

In [84]:
reps = 1
steps_per_epoch = 3
epochs = 4
alignment_dims = [(reps, steps_per_epoch, 1), (reps, steps_per_epoch, 1),
                  (reps, steps_per_epoch, 5), (reps, steps_per_epoch, 10)]

n_layers = len(alignment_dims)
n_processes = 3
print(f'expected alignment dimensions are: {alignment_dims}')
full_alignment = [[[torch.zeros(layer_dims, dtype=torch.float, device='cpu')
                    for proc in range(n_processes)]
                    for layer_dims in alignment_dims]
                    for step in range(epochs)]

expected alignment dimensions are: [(1, 3, 1), (1, 3, 1), (1, 3, 5), (1, 3, 10)]


In [73]:
for i in range(epochs):
    full_alignment.extend(full_alignment)

In [89]:
full_alignment = [[torch.zeros(layer_dims, dtype=torch.float, device='cpu')
                    for proc in range(n_processes)]
                    for layer_dims in alignment_dims]
[torch.cat(layer, axis=1).shape for layer in full_alignment]


[torch.Size([1, 9, 1]),
 torch.Size([1, 9, 1]),
 torch.Size([1, 9, 5]),
 torch.Size([1, 9, 10])]

In [66]:
cat_processes = [[torch.cat(i_step, axis=1).shape for i_step in layer] for layer in full_alignment]
# print([step for step in cat_processes])

In [110]:
tmp2 = [torch.cat(ilayer, axis=1) for ilayer in zip(*cat_processes)]
[layer.shape for layer in tmp2]

In [123]:
n_processes = 3

split_interval = [len(layer[1]) // n_processes for layer in cat_processes]
print(split_interval)
split_list = [[layer[0, :, :][int(i):int(i+split_interval)] for i in torch.arange(len(layer))[::layer_split]] for layer, layer_split in zip(tmp2, split_interval)]

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [107]:
tmp = torch.arange(36)
n_processes = 3

split_interval = len(tmp) // n_processes
split_list = [tmp[int(i):int(i+split_interval)] for i in torch.arange(len(tmp))[::split_interval]]

In [108]:
permuted_list = [torch.tensor(pair) for pair in zip(*split_list)]

In [70]:
[[torch.cat(layer) for layer in step] for step in cat_processes]

TypeError: cat(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [ ]:
(n_layers, n_proc, layer_dims)

In [1]:
import sys
sys.path.append('/Users/celiaberon/GitHub/networkAlignmentAnalysis/')
from networkAlignmentAnalysis.utils import get_list_dims
import torch

In [2]:
a = [[[0, 0, 0], [0, 0, 0]], [[1, 1, 1], [1, 1, 1]]]
b = get_list_dims(a)
b

((2, (2, (3,), (3,)), (2, (3,), (3,))), 18)

In [3]:
def get_dimensions(mixed_structure, preserve_arrays=True):
    """
    A function to determine the dimensions of a mixed structure consisting
    of nested lists and/or numpy arrays.

    Args:
    - mixed_structure: A nested structure potentially containing lists and numpy arrays

    Returns:
    - A tuple representing the dimensions of the mixed structure
    """
    if isinstance(mixed_structure, torch.Tensor):
        # Convert numpy array shape to a tuple and return
        if preserve_arrays:
            return (mixed_structure.shape,)
        else:
            return mixed_structure.shape
    elif isinstance(mixed_structure, list):
        # If it's a list, process each item
        dimensions = []
        for item in mixed_structure:
            item_dims = get_dimensions(item)
            # Only add item dimensions if they provide additional info
            if item_dims:
                dimensions.append(item_dims)
        # Return a tuple with the length of the list and the dimensions found within
        return (len(mixed_structure),) + tuple(dimensions)
    else:
        # Base case: reached an item that is neither a list nor an array
        return ()

In [2]:
a = [[torch.zeros(3,2), torch.zeros(3,2)], [torch.zeros(3,2), torch.zeros(3,2)]]
b = get_dimensions(a, preserve_arrays=False)
b

NameError: name 'get_dimensions' is not defined

In [6]:
a = [[[0, 0, 0], [0, 0, 0]], [[1, 1, 1], [1, 1, 1]]]
b = get_dimensions(a)
b[0]

2

In [67]:

import numpy as np

def construct_zeros_obj(input, device='cpu'):
    """
    Construct an object of zeros that matches the dimensions of the given nested list or array.
    Returns a new object with the same structure as the input, filled with zeros.
    """

    if isinstance(input, np.ndarray):
        # For numpy arrays, create a zeros array with the same shape.
        return np.zeros(input.shape, dtype=input.dtype)
    elif isinstance(input, torch.Tensor):
        # Same for tensors.
        return torch.zeros(input.shape, dtype=input.dtype, device=device)
    elif isinstance(input, list):
        # For lists, recursively build a list structure.
        return [construct_zeros_obj(item) for item in input]
    else:
        # Base case: reached a scalar value, replace it with 0.
        return 0

def replicate_dimension(input, target_dim, n_reps, current_dim=0):
    """
    Introduce N replicates of a particular dimension in the given structure.
    
    Args:
    - structure: The structure to replicate a dimension in.
    - target_dim: The target dimension to replicate.
    - n_reps: The number of times to replicate the target dimension.
    - current_dim: The current dimension in the structure (used internally).
    
    Returns:
    - The modified structure with the specified dimension replicated.
    """
    if current_dim == target_dim:
        return torch.cat([input for _ in range(n_reps)], dim=0)
    else:
        return [replicate_dimension(item, target_dim, n_reps, current_dim + 1) for item in input]

a = [[torch.zeros(3,2), torch.zeros(3,2)], [torch.zeros(3,2), torch.zeros(3,2)]]
b = construct_zeros_obj(a)

c = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
d = construct_zeros_obj(c)


Zero structure matching 'a': [[tensor([[0., 0.],
        [0., 0.],
        [0., 0.]]), tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])], [tensor([[0., 0.],
        [0., 0.],
        [0., 0.]]), tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])]]
Zero structure matching 'c':
[[[0 0]
  [0 0]]

 [[0 0]
  [0 0]]]


In [69]:
replicate_dimension(a, 2, 3)

[[tensor([[0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.]]),
  tensor([[0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.]])],
 [tensor([[0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.]]),
  tensor([[0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.]])]]

In [65]:
get_list_dims(replicate_dimension(a, 2, 3))

(2,
 (2, (torch.Size([3, 6]),), (torch.Size([3, 6]),)),
 (2, (torch.Size([3, 6]),), (torch.Size([3, 6]),)))

In [61]:
[torch.cat(v[0]) for v in replicate_dimension(a, 0, 3)]

TypeError: expected Tensor as element 0 in argument 0, but got list

In [4]:
def get_nested_depth(list_of_lists, current_depth=0, target_type=torch.Tensor):

    assert(target_type != list)
    if isinstance(list_of_lists, target_type):
        return current_depth
    elif isinstance(list_of_lists, list):
        for item in list_of_lists:
            depth = get_nested_depth(item, current_depth+1)
        if depth != -1:
            return depth
        
    return -1

In [48]:
a = [[torch.zeros(3,2), torch.zeros(3,2)], [torch.zeros(3,2), torch.zeros(3,2)]]

get_nested_depth(a)

2